# Setup 

### Import libraries

In [63]:
import yaml 
from pathlib import Path
import os
path = Path("C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos").resolve()  # Get the absolute path
os.chdir(path)
import prediction as pred 
from data import *
from simulation import * 
import copy

Load data from UNOS and dictionaries to map ethnicities and blood type compatibility

In [64]:
df_base = load_data()   
yaml_file_path = "C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos/mapping_data.yaml"
with open(yaml_file_path, "r") as file:
    loaded_data = yaml.safe_load(file)
ethcat_mapping = loaded_data["ethcat_mapping"]
blood_type_compatibility = loaded_data["blood_type_compatibility"]

Get proportions by gender from UNOS sample

In [65]:
real_gender_df = df_base.GENDER.value_counts(normalize=True).reset_index()
real_gender_df.columns = ['gender', 'sample_proportion']
real_gender_df

,gender,sample_proportion
0,M,0.64827
1,F,0.35173


Get proportions by ethnicity from UNOS sample

In [66]:
real_ethcat_df = df_base.ETHCAT.value_counts(normalize=True).reset_index()
real_ethcat_df.ETHCAT = real_ethcat_df.ETHCAT.map(ethcat_mapping)
real_ethcat_df.columns = ['ethcat', 'sample_proportion']
real_ethcat_df

,ethcat,sample_proportion
0,"White, Non-Hispanic",0.439416
1,"Black, Non-Hispanic",0.325258
2,Hispanic/Latino,0.159551
3,"Asian, Non-Hispanic",0.053213
4,"Amer Ind/Alaska Native, Non-Hispanic",0.013618
5,"Multiracial, Non-Hispanic",0.004764
6,"Native Hawaiian/Other Pacific Islander, Non-Hi...",0.004180


Load predictors and create events

In [67]:
decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_0912.pkl')
cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')
random_forest = pred.SurvivalPredictionModel(model_path=r'models\forest_0912.pkl')

replicates = 1
events_per_replication =create_list_of_events(n_events = 10000, replications=replicates)

events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\HP\AppData\Local\Temp\ipykernel_25008\1072111550.py:1: SyntaxWarning: invalid escape sequence '\d'
  decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_0912.pkl')
C:\Users\HP\AppData\Local\Temp\ipykernel_25008\1072111550.py:2: SyntaxWarning: invalid escape sequence '\c'
  cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')


models\decision_tree_0912.pkl
ok
Model loaded successfully from models\decision_tree_0912.pkl.
models\coxprop_0912.pkl
ok
Model loaded successfully from models\coxprop_0912.pkl.
models\forest_0912.pkl
ok
Model loaded successfully from models\forest_0912.pkl.


# Survival Decision tree

## Policy 1

In [68]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  replicates, decision_tree, policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10960.513189859903time unit [04:51, 37.62time unit/s]                       


### recipients who receive a kidney

In [69]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [70]:

df_waiting_time_no 

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1006.264,NaN,11.0,NaN,2743.233,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1075.969,NaN,51.0,NaN,2828.951,NaN,0.044,0.053
2,"Black, Non-Hispanic",1111.458,NaN,377.0,NaN,2867.293,NaN,0.326,0.325
3,Hispanic/Latino,1057.861,NaN,207.0,NaN,2860.754,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1060.686,NaN,6.0,NaN,2849.540,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",945.413,NaN,4.0,NaN,3098.657,NaN,0.003,0.004
6,"White, Non-Hispanic",1112.899,NaN,501.0,NaN,2835.269,NaN,0.433,0.439


In [71]:

df_gender_no.round(3)

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.750,NaN,435.0,NaN,2837.499,NaN,0.376,0.352
1,M,1087.247,NaN,722.0,NaN,2857.683,NaN,0.624,0.648


### patients who leave the waitlist

In [72]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [73]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",712.346,NaN,28.0,NaN,0.0,NaN,0.063,0.053
2,"Black, Non-Hispanic",552.959,NaN,147.0,NaN,0.0,NaN,0.329,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",571.267,NaN,184.0,NaN,0.0,NaN,0.412,0.439


In [74]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,525.625526,NaN,151.0,NaN,0.0,NaN,0.337808,0.35173
1,M,584.630073,NaN,296.0,NaN,0.0,NaN,0.662192,0.64827


### patients who stay in final list

In [75]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [76]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",677.536,NaN,4.0,NaN,0.0,NaN,0.018,0.014
1,"Asian, Non-Hispanic",419.736,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",658.444,NaN,79.0,NaN,0.0,NaN,0.359,0.325
3,Hispanic/Latino,530.628,NaN,41.0,NaN,0.0,NaN,0.186,0.160
4,"Multiracial, Non-Hispanic",376.209,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",959.782,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",627.746,NaN,83.0,NaN,0.0,NaN,0.377,0.439


In [77]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,621.635282,NaN,77.0,NaN,0.0,NaN,0.35,0.35173
1,M,605.496377,NaN,143.0,NaN,0.0,NaN,0.65,0.64827


In [78]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.550000
"Asian, Non-Hispanic",0.579545
"Black, Non-Hispanic",0.625207
Hispanic/Latino,0.627273
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.652344


## Policy 2

In [79]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, decision_tree, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10960.513189859903time unit [01:40, 109.06time unit/s]                      


### recipients who receive a kidney

In [80]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [81]:
df_waiting_time_no 

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1006.264,NaN,11.0,NaN,2743.233,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1075.969,NaN,51.0,NaN,2828.951,NaN,0.044,0.053
2,"Black, Non-Hispanic",1111.458,NaN,377.0,NaN,2867.293,NaN,0.326,0.325
3,Hispanic/Latino,1057.861,NaN,207.0,NaN,2860.754,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1060.686,NaN,6.0,NaN,2849.540,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",945.413,NaN,4.0,NaN,3098.657,NaN,0.003,0.004
6,"White, Non-Hispanic",1112.899,NaN,501.0,NaN,2835.269,NaN,0.433,0.439


In [82]:
df_gender_no.round(3)

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.750,NaN,435.0,NaN,2837.499,NaN,0.376,0.352
1,M,1087.247,NaN,722.0,NaN,2857.683,NaN,0.624,0.648


### patients who leave the waitlist

In [83]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [84]:
df_waiting_time_ll 

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",712.346,NaN,28.0,NaN,0.0,NaN,0.063,0.053
2,"Black, Non-Hispanic",552.959,NaN,147.0,NaN,0.0,NaN,0.329,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",571.267,NaN,184.0,NaN,0.0,NaN,0.412,0.439


In [85]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,525.625526,NaN,151.0,NaN,0.0,NaN,0.337808,0.35173
1,M,584.630073,NaN,296.0,NaN,0.0,NaN,0.662192,0.64827


### patients who stay in final list

In [86]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [87]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",677.536,NaN,4.0,NaN,0.0,NaN,0.018,0.014
1,"Asian, Non-Hispanic",419.736,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",658.444,NaN,79.0,NaN,0.0,NaN,0.359,0.325
3,Hispanic/Latino,530.628,NaN,41.0,NaN,0.0,NaN,0.186,0.160
4,"Multiracial, Non-Hispanic",376.209,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",959.782,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",627.746,NaN,83.0,NaN,0.0,NaN,0.377,0.439


In [88]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,621.635282,NaN,77.0,NaN,0.0,NaN,0.35,0.35173
1,M,605.496377,NaN,143.0,NaN,0.0,NaN,0.65,0.64827


In [89]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.550000
"Asian, Non-Hispanic",0.579545
"Black, Non-Hispanic",0.625207
Hispanic/Latino,0.627273
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.652344


## Policy 3

In [90]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.129074162429time unit [18:25,  9.91time unit/s]                        


### recipients who receive a kidney

In [91]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [92]:
df_waiting_time_no 

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",972.116,NaN,13.0,NaN,2962.896,NaN,0.011,0.014
1,"Asian, Non-Hispanic",1055.303,NaN,53.0,NaN,2961.396,NaN,0.045,0.053
2,"Black, Non-Hispanic",1074.137,NaN,381.0,NaN,2977.524,NaN,0.325,0.325
3,Hispanic/Latino,1030.199,NaN,209.0,NaN,2962.479,NaN,0.178,0.160
4,"Multiracial, Non-Hispanic",1024.303,NaN,6.0,NaN,2922.262,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",923.675,NaN,4.0,NaN,3029.556,NaN,0.003,0.004
6,"White, Non-Hispanic",1059.486,NaN,505.0,NaN,2960.513,NaN,0.431,0.439


In [93]:
df_gender_no.round(3)

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1071.448,NaN,441.0,NaN,2969.812,NaN,0.377,0.352
1,M,1048.628,NaN,730.0,NaN,2964.507,NaN,0.623,0.648


### patients who leave the waitlist

In [94]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [95]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",677.941,NaN,26.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",538.288,NaN,147.0,NaN,0.0,NaN,0.328,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",569.313,NaN,187.0,NaN,0.0,NaN,0.417,0.439


In [96]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,507.942756,NaN,149.0,NaN,0.0,NaN,0.332589,0.35173
1,M,580.632089,NaN,299.0,NaN,0.0,NaN,0.667411,0.64827


### patients who stay in final list

In [97]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [98]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",479.357,NaN,2.0,NaN,0.0,NaN,0.009,0.014
1,"Asian, Non-Hispanic",449.438,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",624.037,NaN,82.0,NaN,0.0,NaN,0.378,0.325
3,Hispanic/Latino,540.594,NaN,39.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",406.600,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",966.553,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",609.918,NaN,81.0,NaN,0.0,NaN,0.373,0.439


In [99]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,587.532241,NaN,78.0,NaN,0.0,NaN,0.359447,0.35173
1,M,597.266499,NaN,139.0,NaN,0.0,NaN,0.640553,0.64827


In [100]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.650000
"Asian, Non-Hispanic",0.602273
"Black, Non-Hispanic",0.624590
Hispanic/Latino,0.633333
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.653299


# Random survival forest

In [101]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [102]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, random_forest, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10960.431973413146time unit [09:39, 18.91time unit/s]                      


### recipients who receive a kidney

In [103]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [104]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",481.429,NaN,11.0,NaN,2845.445,NaN,0.009,0.014
1,"Asian, Non-Hispanic",690.032,NaN,59.0,NaN,2981.202,NaN,0.050,0.053
2,"Black, Non-Hispanic",735.329,NaN,387.0,NaN,2957.384,NaN,0.327,0.325
3,Hispanic/Latino,703.724,NaN,188.0,NaN,2988.301,NaN,0.159,0.160
4,"Multiracial, Non-Hispanic",1259.012,NaN,4.0,NaN,2899.847,NaN,0.003,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",2181.947,NaN,6.0,NaN,2967.649,NaN,0.005,0.004
6,"White, Non-Hispanic",648.584,NaN,528.0,NaN,2973.877,NaN,0.446,0.439


In [105]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,696.436527,NaN,450.0,NaN,2963.635873,NaN,0.380389,0.35173
1,M,695.857874,NaN,733.0,NaN,2973.363491,NaN,0.619611,0.64827


### patients who leave the waitlist

In [106]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [107]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1337.636,NaN,8.0,NaN,0.0,NaN,0.018,0.014
1,"Asian, Non-Hispanic",1440.706,NaN,25.0,NaN,0.0,NaN,0.055,0.053
2,"Black, Non-Hispanic",1527.629,NaN,157.0,NaN,0.0,NaN,0.348,0.325
3,Hispanic/Latino,1345.040,NaN,98.0,NaN,0.0,NaN,0.217,0.160
4,"Multiracial, Non-Hispanic",3399.425,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",1512.533,NaN,160.0,NaN,0.0,NaN,0.355,0.439


In [108]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1542.281353,NaN,157.0,NaN,0.0,NaN,0.348115,0.35173
1,M,1438.902494,NaN,294.0,NaN,0.0,NaN,0.651885,0.64827


### patients who stay in final list

In [109]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [110]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",530.383,NaN,3.0,NaN,0.0,NaN,0.013,0.014
1,"Asian, Non-Hispanic",2323.560,NaN,10.0,NaN,0.0,NaN,0.044,0.053
2,"Black, Non-Hispanic",1859.096,NaN,79.0,NaN,0.0,NaN,0.345,0.325
3,Hispanic/Latino,1845.452,NaN,51.0,NaN,0.0,NaN,0.223,0.160
4,"Multiracial, Non-Hispanic",1076.503,NaN,5.0,NaN,0.0,NaN,0.022,0.005
5,"White, Non-Hispanic",1593.346,NaN,81.0,NaN,0.0,NaN,0.354,0.439


In [111]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1788.413774,NaN,73.0,NaN,0.0,NaN,0.318777,0.35173
1,M,1728.863641,NaN,156.0,NaN,0.0,NaN,0.681223,0.64827


In [112]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.500000
"Asian, Non-Hispanic",0.627660
"Black, Non-Hispanic",0.621188
Hispanic/Latino,0.557864
"Multiracial, Non-Hispanic",0.400000
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.750000
"White, Non-Hispanic",0.686606


## Policy 2

In [113]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, random_forest, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10960.513189859903time unit [01:30, 121.39time unit/s]                      


### recipients who receive a kidney

In [114]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [115]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1006.264,NaN,11.0,NaN,2845.313,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1075.969,NaN,51.0,NaN,2944.118,NaN,0.044,0.053
2,"Black, Non-Hispanic",1111.458,NaN,377.0,NaN,2964.505,NaN,0.326,0.325
3,Hispanic/Latino,1057.861,NaN,207.0,NaN,2953.848,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1060.686,NaN,6.0,NaN,2850.898,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",945.413,NaN,4.0,NaN,3047.687,NaN,0.003,0.004
6,"White, Non-Hispanic",1112.899,NaN,501.0,NaN,2968.131,NaN,0.433,0.439


In [116]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.749763,NaN,435.0,NaN,2956.138878,NaN,0.375972,0.35173
1,M,1087.247231,NaN,722.0,NaN,2965.267025,NaN,0.624028,0.64827


### patients who leave the waitlist

In [117]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [118]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",712.346,NaN,28.0,NaN,0.0,NaN,0.063,0.053
2,"Black, Non-Hispanic",552.959,NaN,147.0,NaN,0.0,NaN,0.329,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",571.267,NaN,184.0,NaN,0.0,NaN,0.412,0.439


In [119]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,525.625526,NaN,151.0,NaN,0.0,NaN,0.337808,0.35173
1,M,584.630073,NaN,296.0,NaN,0.0,NaN,0.662192,0.64827


### patients who stay in final list

In [120]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [121]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",677.536,NaN,4.0,NaN,0.0,NaN,0.018,0.014
1,"Asian, Non-Hispanic",419.736,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",658.444,NaN,79.0,NaN,0.0,NaN,0.359,0.325
3,Hispanic/Latino,530.628,NaN,41.0,NaN,0.0,NaN,0.186,0.160
4,"Multiracial, Non-Hispanic",376.209,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",959.782,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",627.746,NaN,83.0,NaN,0.0,NaN,0.377,0.439


In [122]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,621.635282,NaN,77.0,NaN,0.0,NaN,0.35,0.35173
1,M,605.496377,NaN,143.0,NaN,0.0,NaN,0.65,0.64827


In [123]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.550000
"Asian, Non-Hispanic",0.579545
"Black, Non-Hispanic",0.625207
Hispanic/Latino,0.627273
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.652344


## Policy 3

### recipients who receive a kidney

In [124]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.129074162429time unit [15:38, 11.68time unit/s]                        


In [125]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [126]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",972.116,NaN,13.0,NaN,2962.896,NaN,0.011,0.014
1,"Asian, Non-Hispanic",1055.303,NaN,53.0,NaN,2961.396,NaN,0.045,0.053
2,"Black, Non-Hispanic",1074.137,NaN,381.0,NaN,2977.524,NaN,0.325,0.325
3,Hispanic/Latino,1030.199,NaN,209.0,NaN,2962.479,NaN,0.178,0.160
4,"Multiracial, Non-Hispanic",1024.303,NaN,6.0,NaN,2922.262,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",923.675,NaN,4.0,NaN,3029.556,NaN,0.003,0.004
6,"White, Non-Hispanic",1059.486,NaN,505.0,NaN,2960.513,NaN,0.431,0.439


In [127]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1071.448187,NaN,441.0,NaN,2969.81213,NaN,0.376601,0.35173
1,M,1048.628252,NaN,730.0,NaN,2964.50724,NaN,0.623399,0.64827


### patients who leave the waitlist

In [128]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [129]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",677.941,NaN,26.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",538.288,NaN,147.0,NaN,0.0,NaN,0.328,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",569.313,NaN,187.0,NaN,0.0,NaN,0.417,0.439


In [130]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,507.942756,NaN,149.0,NaN,0.0,NaN,0.332589,0.35173
1,M,580.632089,NaN,299.0,NaN,0.0,NaN,0.667411,0.64827


### patients who stay in final list

In [131]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [132]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",479.357,NaN,2.0,NaN,0.0,NaN,0.009,0.014
1,"Asian, Non-Hispanic",449.438,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",624.037,NaN,82.0,NaN,0.0,NaN,0.378,0.325
3,Hispanic/Latino,540.594,NaN,39.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",406.600,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",966.553,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",609.918,NaN,81.0,NaN,0.0,NaN,0.373,0.439


In [133]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,587.532241,NaN,78.0,NaN,0.0,NaN,0.359447,0.35173
1,M,597.266499,NaN,139.0,NaN,0.0,NaN,0.640553,0.64827


# Cox proportional hazard models

In [134]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [135]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.101687856248time unit [03:08, 58.19time unit/s]                       


### recipients who receive a kidney

In [136]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [137]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",47.208,NaN,18.0,NaN,2634.631,NaN,0.015,0.014
1,"Asian, Non-Hispanic",249.068,NaN,58.0,NaN,2626.023,NaN,0.049,0.053
2,"Black, Non-Hispanic",194.915,NaN,422.0,NaN,2668.461,NaN,0.357,0.325
3,Hispanic/Latino,322.633,NaN,213.0,NaN,2619.289,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",397.874,NaN,7.0,NaN,2446.352,NaN,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",40.889,NaN,3.0,NaN,2666.975,NaN,0.003,0.004
6,"White, Non-Hispanic",288.259,NaN,461.0,NaN,2642.290,NaN,0.390,0.439


In [138]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,203.933522,NaN,431.0,NaN,2648.964746,NaN,0.364636,0.35173
1,M,285.180303,NaN,751.0,NaN,2643.473593,NaN,0.635364,0.64827


### patients who leave the waitlist

In [139]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [140]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1836.527,NaN,1.0,NaN,0.0,NaN,0.002,0.014
1,"Asian, Non-Hispanic",1860.309,NaN,26.0,NaN,0.0,NaN,0.060,0.053
2,"Black, Non-Hispanic",2403.125,NaN,109.0,NaN,0.0,NaN,0.251,0.325
3,Hispanic/Latino,2025.808,NaN,80.0,NaN,0.0,NaN,0.184,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",5814.015,NaN,3.0,NaN,0.0,NaN,0.007,0.004
5,"White, Non-Hispanic",2456.671,NaN,216.0,NaN,0.0,NaN,0.497,0.439


In [141]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,2472.507135,NaN,143.0,NaN,0.0,NaN,0.328736,0.35173
1,M,2290.151079,NaN,292.0,NaN,0.0,NaN,0.671264,0.64827


### patients who stay in final list

In [142]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [143]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",10.651,NaN,1.0,NaN,0.0,NaN,0.004,0.014
1,"Asian, Non-Hispanic",2782.059,NaN,8.0,NaN,0.0,NaN,0.034,0.053
2,"Black, Non-Hispanic",2384.022,NaN,71.0,NaN,0.0,NaN,0.305,0.325
3,Hispanic/Latino,1965.757,NaN,45.0,NaN,0.0,NaN,0.193,0.160
4,"Multiracial, Non-Hispanic",210.848,NaN,3.0,NaN,0.0,NaN,0.013,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",3627.735,NaN,2.0,NaN,0.0,NaN,0.009,0.004
6,"White, Non-Hispanic",2946.610,NaN,103.0,NaN,0.0,NaN,0.442,0.439


In [144]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,2869.459995,NaN,95.0,NaN,0.0,NaN,0.407725,0.35173
1,M,2310.013712,NaN,138.0,NaN,0.0,NaN,0.592275,0.64827


In [145]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.900000
"Asian, Non-Hispanic",0.630435
"Black, Non-Hispanic",0.700997
Hispanic/Latino,0.630178
"Multiracial, Non-Hispanic",0.700000
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.375000
"White, Non-Hispanic",0.591026


## Policy 2

In [146]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10960.513189859903time unit [01:15, 145.52time unit/s]                      


### recipients who receive a kidney

In [147]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [148]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1006.264,NaN,11.0,NaN,2398.834,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1075.969,NaN,51.0,NaN,2468.747,NaN,0.044,0.053
2,"Black, Non-Hispanic",1111.458,NaN,377.0,NaN,2603.841,NaN,0.326,0.325
3,Hispanic/Latino,1057.861,NaN,207.0,NaN,2549.752,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1060.686,NaN,6.0,NaN,2472.503,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",945.413,NaN,4.0,NaN,2620.162,NaN,0.003,0.004
6,"White, Non-Hispanic",1112.899,NaN,501.0,NaN,2557.261,NaN,0.433,0.439


In [149]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.749763,NaN,435.0,NaN,2567.903371,NaN,0.375972,0.35173
1,M,1087.247231,NaN,722.0,NaN,2563.996722,NaN,0.624028,0.64827


### patients who leave the waitlist

In [150]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [151]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",520.094,NaN,5.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",712.346,NaN,28.0,NaN,0.0,NaN,0.063,0.053
2,"Black, Non-Hispanic",552.959,NaN,147.0,NaN,0.0,NaN,0.329,0.325
3,Hispanic/Latino,526.519,NaN,82.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",571.267,NaN,184.0,NaN,0.0,NaN,0.412,0.439


In [152]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,525.625526,NaN,151.0,NaN,0.0,NaN,0.337808,0.35173
1,M,584.630073,NaN,296.0,NaN,0.0,NaN,0.662192,0.64827


### patients who stay in final list

In [153]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [154]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",677.536,NaN,4.0,NaN,0.0,NaN,0.018,0.014
1,"Asian, Non-Hispanic",419.736,NaN,9.0,NaN,0.0,NaN,0.041,0.053
2,"Black, Non-Hispanic",658.444,NaN,79.0,NaN,0.0,NaN,0.359,0.325
3,Hispanic/Latino,530.628,NaN,41.0,NaN,0.0,NaN,0.186,0.160
4,"Multiracial, Non-Hispanic",376.209,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",959.782,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",627.746,NaN,83.0,NaN,0.0,NaN,0.377,0.439


In [155]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,621.635282,NaN,77.0,NaN,0.0,NaN,0.35,0.35173
1,M,605.496377,NaN,143.0,NaN,0.0,NaN,0.65,0.64827


In [156]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.550000
"Asian, Non-Hispanic",0.579545
"Black, Non-Hispanic",0.625207
Hispanic/Latino,0.627273
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.652344


## Policy 3

In [157]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10958.752787439018time unit [09:13, 19.79time unit/s]                      


### recipients who receive a kidney

In [158]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [159]:
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",979.052,NaN,11.0,NaN,2672.080,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1160.981,NaN,51.0,NaN,2581.671,NaN,0.044,0.053
2,"Black, Non-Hispanic",1044.022,NaN,385.0,NaN,2623.228,NaN,0.335,0.325
3,Hispanic/Latino,1042.493,NaN,204.0,NaN,2566.597,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1021.531,NaN,6.0,NaN,2750.869,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1225.432,NaN,4.0,NaN,2633.149,NaN,0.003,0.004
6,"White, Non-Hispanic",1089.124,NaN,489.0,NaN,2571.761,NaN,0.425,0.439


In [160]:
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1085.08950,NaN,433.0,NaN,2611.893771,NaN,0.376522,0.35173
1,M,1057.69287,NaN,717.0,NaN,2577.776870,NaN,0.623478,0.64827


### patients who leave the waitlist

In [161]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [162]:
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",649.747,NaN,7.0,NaN,0.0,NaN,0.016,0.014
1,"Asian, Non-Hispanic",722.444,NaN,26.0,NaN,0.0,NaN,0.059,0.053
2,"Black, Non-Hispanic",530.187,NaN,139.0,NaN,0.0,NaN,0.314,0.325
3,Hispanic/Latino,547.941,NaN,81.0,NaN,0.0,NaN,0.183,0.160
4,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
5,"White, Non-Hispanic",604.528,NaN,189.0,NaN,0.0,NaN,0.427,0.439


In [163]:
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,531.498466,NaN,150.0,NaN,0.0,NaN,0.3386,0.35173
1,M,601.512683,NaN,293.0,NaN,0.0,NaN,0.6614,0.64827


### patients who stay in final list

In [164]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [165]:
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",432.630,NaN,2.0,NaN,0.0,NaN,0.009,0.014
1,"Asian, Non-Hispanic",476.879,NaN,11.0,NaN,0.0,NaN,0.050,0.053
2,"Black, Non-Hispanic",698.075,NaN,76.0,NaN,0.0,NaN,0.349,0.325
3,Hispanic/Latino,576.013,NaN,41.0,NaN,0.0,NaN,0.188,0.160
4,"Multiracial, Non-Hispanic",361.829,NaN,3.0,NaN,0.0,NaN,0.014,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",937.443,NaN,1.0,NaN,0.0,NaN,0.005,0.004
6,"White, Non-Hispanic",625.204,NaN,84.0,NaN,0.0,NaN,0.385,0.439


In [166]:
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,635.334386,NaN,72.0,NaN,0.0,NaN,0.330275,0.35173
1,M,627.240570,NaN,146.0,NaN,0.0,NaN,0.669725,0.64827


In [167]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)

,result
ethcat,
"Amer Ind/Alaska Native, Non-Hispanic",0.550000
"Asian, Non-Hispanic",0.579545
"Black, Non-Hispanic",0.641667
Hispanic/Latino,0.625767
"Multiracial, Non-Hispanic",0.666667
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",0.666667
"White, Non-Hispanic",0.641732
